In [1]:
DIRECTORY = "C:\\Work\\teamwork-visualiser-dashboard\\server\\saved_data"

import pandas as pd

In [59]:
# Load the data

PATH = DIRECTORY + "\\416\\" + 'heart rate-RED.csv'
PATH_HIVE = DIRECTORY + "\\416\\" + "\\hive_data_folder\\result\\416_RED.csv"

df = pd.read_csv(PATH, parse_dates=['Server Time'], index_col='Server Time')
processed_hive_df = pd.read_csv(PATH_HIVE)
processed_hive_df['audio_start_timestamp'] = processed_hive_df['audio_start_timestamp'].astype('int64')
processed_hive_df['audio_in_sec'] = pd.to_timedelta(processed_hive_df['audio time']).dt.total_seconds().astype('int64')
processed_hive_df['timestamp'] = processed_hive_df['audio_start_timestamp'] + processed_hive_df['audio_in_sec']


# Resample the data to 1Hz frequency, taking the last value if there are multiple within one second
hr_df = df.resample('1s').last()

# Drop rows with NaN values that may appear after resampling
hr_df = hr_df.ffill()

# # Save the resampled data to a new CSV file
# resampled_df.to_csv('resampled_data.csv')

hr_df = hr_df.reset_index()
hr_df = hr_df.rename(columns={'Value': 'heartrate'})


hr_df['Server Time'] = hr_df['Server Time'].dt.tz_convert('Australia/Melbourne')
hr_df['hr_timestamp'] = (hr_df['Server Time'] - pd.Timestamp("1970-01-01", tz='UTC')) // pd.Timedelta('1s')

# sort values by timestamp:
hr_df = hr_df.sort_values('hr_timestamp')
merged_df = pd.merge_asof(processed_hive_df, hr_df, left_on='timestamp', right_on='hr_timestamp', direction='nearest')


merged_df

,audio_start_timestamp,audio time,x,y,audio,tagId,audio_in_sec,timestamp,Server Time_x,Watch Timestamp_x,Value,hr_timestamp_x,Server Time_y,Watch Timestamp_y,heartrate,hr_timestamp_y
0,1719455383,0:00:01,10333.517441,6202.103198,1,RED,1,1719455384,2024-06-27 12:29:44+10:00,317441067.0,91.0,1719455384,2024-06-27 12:29:44+10:00,317441067.0,91.0,1719455384
1,1719455383,0:00:02,10282.487704,6248.930328,1,RED,2,1719455385,2024-06-27 12:29:45+10:00,317442052.0,91.0,1719455385,2024-06-27 12:29:45+10:00,317442052.0,91.0,1719455385
2,1719455383,0:00:03,10262.344968,6277.943239,1,RED,3,1719455386,2024-06-27 12:29:46+10:00,317443038.0,91.0,1719455386,2024-06-27 12:29:46+10:00,317443038.0,91.0,1719455386
3,1719455383,0:00:04,10179.480797,6346.817923,1,RED,4,1719455387,2024-06-27 12:29:47+10:00,317444019.0,91.0,1719455387,2024-06-27 12:29:47+10:00,317444019.0,91.0,1719455387
4,1719455383,0:00:05,10230.338030,6359.267606,1,RED,5,1719455388,2024-06-27 12:29:48+10:00,317445006.0,90.0,1719455388,2024-06-27 12:29:48+10:00,317445006.0,90.0,1719455388
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1473,1719455383,0:24:34,4804.723488,4715.904700,0,RED,1474,1719456857,2024-06-27 12:54:17+10:00,318914045.0,95.0,1719456857,2024-06-27 12:54:17+10:00,318914045.0,95.0,1719456857
1474,1719455383,0:24:35,4807.383783,4747.621625,1,RED,1475,1719456858,2024-06-27 12:54:18+10:00,318915032.0,95.0,1719456858,2024-06-27 12:54:18+10:00,318915032.0,95.0,1719456858
1475,1719455383,0:24:36,4779.838489,4787.597580,0,RED,1476,1719456859,2024-06-27 12:54:19+10:00,318916016.0,95.0,1719456859,2024-06-27 12:54:19+10:00,318916016.0,95.0,1719456859
1476,1719455383,0:24:37,4709.353489,4835.076617,0,RED,1477,1719456860,2024-06-27 12:54:20+10:00,318917003.0,95.0,1719456860,2024-06-27 12:54:20+10:00,318917003.0,95.0,1719456860
